# XML example and exercise
****
+ study examples of accessing nodes in XML tree structure  
+ work on exercise to be completed and submitted
****
+ reference: https://docs.python.org/2.7/library/xml.etree.elementtree.html
+ data source: http://www.dbis.informatik.uni-goettingen.de/Mondial
****

## XML example

+ for details about tree traversal and iterators, see https://docs.python.org/2.7/library/xml.etree.elementtree.html

In [1]:
# import libraries

import pandas as pd
from xml.etree import ElementTree as ET
import numpy as np

In [2]:
document_tree = ET.parse( './data/mondial_database_less.xml' )

In [3]:
# print names of all countries
for child in document_tree.getroot():
    print (child.find('name').text)

Albania
Greece
Macedonia
Serbia
Montenegro
Kosovo
Andorra


In [4]:
# print names of all countries and their cities
for element in document_tree.iterfind('country'):
    print ('* ' + element.find('name').text + ':'),
    capitals_string = ''
    for subelement in element.getiterator('city'):
        capitals_string += subelement.find('name').text + ', '
    print (capitals_string[:-2])

* Albania:
Tirana, Shkodër, Durrës, Vlorë, Elbasan, Korçë
* Greece:
Komotini, Kavala, Athina, Peiraias, Peristeri, Acharnes, Patra, Kozani, Kerkyra, Ioannina, Thessaloniki, Iraklio, Chania, Ermoupoli, Rhodes, Tripoli, Lamia, Chalkida, Larissa, Volos, Mytilini, Karyes
* Macedonia:
Skopje, Kumanovo
* Serbia:
Beograd, Novi Sad, Niš
* Montenegro:
Podgorica
* Kosovo:
Prishtine
* Andorra:
Andorra la Vella


****
## XML exercise

Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find

1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [5]:
# parse the xml document ./data/mondial_database.xml
xml_document = ET.parse( './data/mondial_database.xml' )


In [6]:
# create a shared lookup table for country codes to be used later
country_lookup = []

for country in xml_document.findall('country'):
    for node in list(country):
        if node.tag == 'name':   
            co = node.text
            code = country.attrib['car_code']
            
    country_lookup.append({'code':code, 'country':co}) 
        
countries = pd.DataFrame(country_lookup)
countries.head(10)

,code,country
0,AL,Albania
1,GR,Greece
2,MK,Macedonia
3,SRB,Serbia
4,MNE,Montenegro
5,KOS,Kosovo
6,AND,Andorra
7,F,France
8,E,Spain
9,A,Austria


# Part 1: 10 countries with the lowest infant mortality rates 

In [7]:
# find all countries in the xml document with corresponding infant mortality 
    
im_data=[]
for child in xml_document.getroot().findall('country'):
    # if infant mortality values do not exist, ignore the country
    if child.find('infant_mortality') != None:
        # create a table with country and infant mortality
        im_data.append({'country': child.find('name').text, 'infant_mortality':float(child.find('infant_mortality').text)})

# create a dataFrame with the list of all countries and their infant mortality
im_df = pd.DataFrame(im_data)

# sort the top 10 entries in ascending order of infant mortality and display along with country name
im_df.set_index('country').sort_values('infant_mortality').reset_index()[:10]


,country,infant_mortality
0,Monaco,1.81
1,Japan,2.13
2,Bermuda,2.48
3,Norway,2.48
4,Singapore,2.53
5,Sweden,2.60
6,Czech Republic,2.63
7,Hong Kong,2.73
8,Macao,3.13
9,Iceland,3.15


# Part 2: 10 cities with the largest population

In [8]:
# find all countries in the xml document with cities and corresponding population for given year

pop_data = []      
for child in xml_document.getroot().findall('country'):
    for subchild in child.getiterator('city'):
        # cities have multiple entries according to year
        for elementpop in subchild.findall('population'):
            year = elementpop.attrib['year']
            pop =int(elementpop.text)
        
        # append the data as part of the city loop so that only latest available figures are used
        pop_data.append({'city':subchild.find('name').text,'population': pop, 'year':elementpop.attrib['year'] })

# create a dataFrame with the list of all cities and their populations sorted by year
pop_df = pd.DataFrame(pop_data)

# sort the top 10 entries in descending order of populations and display along with city names and corresponding year
pop_df.set_index('city').sort_values('population', ascending=False).reset_index()[:10]


,city,population,year
0,Shanghai,22315474,2010
1,Istanbul,13710512,2012
2,Mumbai,12442373,2011
3,Moskva,11979529,2013
4,Beijing,11716620,2010
5,São Paulo,11152344,2010
6,Tianjin,11090314,2010
7,Guangzhou,11071424,2010
8,Delhi,11034555,2011
9,Shenzhen,10358381,2010


# Part 3: 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)

In [9]:
# find all ethnic group populations for countries in the xml_document

eth_data = [] 

for country in xml_document.findall('country'):
    for child in list(country):
        # find the total population of the country
        if child.tag == 'population':
            total_pop = int(child.text)
        # find population percentage of individual ethnic communities
        elif child.tag == 'ethnicgroup':
            eth_perc = float(child.attrib['percentage'])
             # calculate total population for each ethnicity based on total population of the country
            eth_pop = int(total_pop * eth_perc / 100.)
            eth_data.append({'ethnicity':child.text, 'population':eth_pop})
            
# create a dataFrame with the list of ethnicity and corresponding population
eth_df = pd.DataFrame(eth_data)
eth_df.head(2)

,ethnicity,population
0,Albanian,2660131
1,Greek,84004


In [10]:
# find the top 10 overall population as the sum of latest/best populations
eth_df.groupby('ethnicity').population.sum().sort_values(ascending=False).head(10).reset_index()

,ethnicity,population
0,Han Chinese,1245058800
1,Indo-Aryan,871815583
2,European,494872201
3,African,318325104
4,Dravidian,302713744
5,Mestizo,157734349
6,Bengali,146776916
7,Russian,131856989
8,Japanese,126534212
9,Malay,121993548


# Part 4: name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [11]:
# find all rivers in the document along with the country and length
r_length=[]
r_name=[]
r_country=[]

for element in xml_document.iterfind('river'):
    try:
        r_length.append(float(element.find('length').text))
    except:
        r_length.append(np.nan)
    r_name.append(element.find('name').text)
    r_country.append(element.attrib['country'].split(' ')[0])
    
river_df = pd.DataFrame({'river_name':r_name,'river_length':r_length,'river_country':r_country})
river_df.head(5)

,river_country,river_length,river_name
0,IS,230.0,Thjorsa
1,IS,206.0,Joekulsa a Fjoellum
2,N,604.0,Glomma
3,N,322.0,Lagen
4,S,93.0,Goetaaelv


In [12]:
# merge the table of rivers along with country names from the lookup table 
df = pd.merge(river_df, countries, how='left', left_on='river_country', right_on='code')

# display the longest river 
df.sort_values('river_length', ascending=False).head(1)

,river_country,river_length,river_name,code,country
174,CO,6448.0,Amazonas,CO,Colombia


In [13]:
# display the longest river in the world 
rl = df.groupby(['country', 'river_name']).river_length.max().sort_values(ascending=False).head(1).reset_index()
print('The longest river in the world is ' + rl['river_name'][0] +
      ' in ' + rl['country'][0] + ' with a length of ' + str('{:5,.0f}'.format(int(rl['river_length'][0]))) + ' kilometers')

The longest river in the world is Amazonas in Colombia with a length of 6,448 kilometers


In [14]:
# find all lakes in the document along with the country and area
lake_data = []

for element in xml_document.iterfind('lake'):
    l_country = element.attrib['country'].split(' ')[0]
    for subelement in list(element):
        if subelement.tag == 'name':
            l_name = element.find('name').text
        elif subelement.tag == 'area':
            l_area = float(element.find('area').text)
        
            lake_data.append({'lake_name':l_name, 'lake_country':l_country, 'lake_area':l_area}) 
    
lake_df = pd.DataFrame(lake_data)
lake_df.head(5)


,lake_area,lake_country,lake_name
0,1040.0,SF,Inari
1,928.0,SF,Oulujaervi
2,472.0,SF,Kallavesi
3,4370.0,SF,Saimaa
4,1118.0,SF,Paeijaenne


In [15]:
# merge the table of lakes along with country names from the lookup table 
df = pd.merge(lake_df, countries, how='left', left_on='lake_country', right_on='code')

# display the largest lake
df.sort_values('lake_area', ascending=False).head(1)

,lake_area,lake_country,lake_name,code,country
54,386400.0,R,Caspian Sea,R,Russia


In [16]:
# display the largest lake in the world 
ll = df.groupby(['country', 'lake_name']).lake_area.max().sort_values(ascending=False).head(1).reset_index()
print('The largest lake in the world is ' + ll['lake_name'][0] +
      ' in ' + ll['country'][0] +
      ' with an area of ' + str('{:5,.0f}'.format(int(ll['lake_area'][0]))) + ' square kilometers')

The largest lake in the world is Caspian Sea in Russia with an area of 386,400 square kilometers


In [17]:
# find all airports in the document along with their corresponding elevation and country
ap_data = []

for element in xml_document.iterfind('airport'):
    a_country = element.attrib['country'].split(' ')[0]
    for subelement in list(element):
        if subelement.tag == 'name':
            a_name = subelement.text
        elif subelement.tag == 'elevation':
            if subelement.text != None: 
                a_elevation = int(subelement.text)
                
                ap_data.append({'airport_name':a_name, 'airport_country':a_country, 'airport_elevation':a_elevation}) 
    
ap_df = pd.DataFrame(ap_data)
ap_df.head(5)

,airport_country,airport_elevation,airport_name
0,AFG,977,Herat
1,AFG,1792,Kabul Intl
2,AL,38,Tirana Rinas
3,DZ,811,Cheikh Larbi Tebessi
4,DZ,822,Batna Airport


In [18]:
# merge the table of airport elevations along with country names from the lookup table 
df = pd.merge(ap_df, countries, how='left', left_on='airport_country', right_on='code')

# display the airport t highest elevation
df.sort_values('airport_elevation', ascending=False).head(1)

,airport_country,airport_elevation,airport_name,code,country
80,BOL,4063,El Alto Intl,BOL,Bolivia


In [19]:
# display the airport at the highest elevation in the world 
ae = df.groupby(['country', 'airport_name']).airport_elevation.max().sort_values(ascending=False).head(1).reset_index()
print('The airport at highest elevation in the world is the ' + ae['airport_name'][0] + ' at an elevation of ' 
       + str('{:5,.0f}'.format(int(ae['airport_elevation'][0]))) + ' meters')
    

The airport at highest elevation in the world is the El Alto Intl at an elevation of 4,063 meters
